#Design of parametric Cottage

##Examples and reference models

<table>
 <tbody>
    <tr>
        <td><img src="img/house_plan0.jpg" style="width: 500px; height: 400px"></td>
        <td><img src="img/house_plan1.gif" style="width: 500px; height: 400px"></td>
    </tr>
</tbody>
</table>

##Previous Workshops modification

In order to accomplish the goals of this workshop some modification to the previous workshops were needed, a link to a clarifying notebook is provided where deep modification was made, texture/color changes are not taken in count:

###[workshop_03 (stair)](src/workshop_03.ipynb)
> this workshop was completely revised, due to the fact that it was made long ago by an unexperienced author, this plugin provide now a model of a more elegant spiral stair as a replacement of the ugly quarter turn stair made in the previous workshop. The new stair fulfill the requirements of its own workshop differently, the plugin accept no more dx, dy and dz as parameter but takes the values of riser, inter step height and stair height in order to accomplish a better parameterization of the stair, mantaining its elegance and with the possibility of an easier insertion in the spatial frame.

###workshop_07 (doors and windows)
> this workshop was barely touched, just some minor fixing to allow a better scaling from px (inkscape) to m (workshop requirements).

###[workshop_08 (house spatial frame)](src/workshop_08.ipynb)
> this workshop was enhanced to manage the possibility of inserting windows and doors model directly inside the spatial frame, two specialized method was created to manage doors and windows respectively. Moreover the main function take ulterior arguments (no arguments in the previous workshop), one for the current story that it's going to be built and three optional parameter, a generating function for the windows, a generating function for the doors and a model of a box of the stair.

###workshop_09 (roof)
> absolutely no changes.

##Variables used in the workshop

No particular variables were used in this workshop (the actual parameter will be described below in the results section).
For windows and doors model were used the same cells used in the 7th workshop, all the variable behaviour of the main function rely on .lines file's variability, enhancing parameterization. By the way some file are required:

###walls, doors and windows

* **muriesterniX.lines**
> X is the story's identifier (e.g. 0 for ground floor, 1 for 1st floor, ecc...)
* **muriinterniX.lines**
> X is the story's identifier (e.g. 0 for ground floor, 1 for 1st floor, ecc...)
* **terrace_wallsX.lines**
> X is the story's identifier (e.g. 0 for ground floor, 1 for 1st floor, ecc...)
* **porte_modelX.lines**
> X is the story's identifier (e.g. 0 for ground floor, 1 for 1st floor, ecc...)
* **finestre_modelX.lines**
> X is the story's identifier (e.g. 0 for ground floor, 1 for 1st floor, ecc...)

###surfaces

* **livingroomXY/bathroomXY/bedroomXY/terrace_surfaceXY.lines**
> X is the story's identifier (e.g. 0 for ground floor, 1 for 1st floor, ecc...), Y is the identifier of a room/surface inside/outside the current story

###ladder/stair

* **ladder.lines**
> used for the stair positioning




##Main function

```python
def multistorey_house(nStorey):
	"""
	buildMultiStorey accept the number of storey in the building.
	"""
	def renderWindows(XWindow, YWindow, occurrencyWindow, windowModel = False):
		"""
		renderWindows accept the window's cells and the occurrency, and optionally a window generating function 
		"""
		def renderDoors(XDoor, YDoor, occurrencyDoor, doorModel = False):
			"""
			renderWindows accept the door's cells and the occurrency, and optionally a door generating function 
			"""
			def renderRoof(vertices, pitchAngle, height):
				"""
				renderRoof accept the vertices of the base roof, a pitch angle and the desired height 
				of the roof
				"""
				def renderLadder(ladderHeight, interStep, riser):
					"""
					renderLadder is the inner function used to assembly all together, it takes the 
					desired height of the ladder, an interstep between two step and a riser for the single
					step.
					"""

					#building the ladder model and the ladder box
					ladderModel = ladder.make_ladder(ladderHeight, interStep, riser)
					with open("lines/ladder.lines", "rb") as ladderFile:
						reader = csv.reader(ladderFile, delimiter=",")
						row = next(reader)
						ladderModel = T([1,2])([float(row[0])*xfactor, float(row[1])*yfactor])(ladderModel)
					ladderBOX = CUBOID([SIZE([1])(ladderModel)[0]/xfactor,SIZE([2])(ladderModel)[0]/yfactor, SIZE([3])(ladderModel)[0]/zfactor])
					ladderBOX = T([1,2])([float(row[0])-SIZE([1])(ladderBOX)[0]/2., float(row[1])-SIZE([2])(ladderBOX)[0]/2.])(ladderBOX)

					#building roof model
					if isinstance(vertices, basestring):
						with open("lines/" + vertices + ".lines", "rb") as file:
							reader = csv.reader(file, delimiter=",")
							newVertices = []
							for row in reader:
								newVertices.append([float(row[0]), float(row[1])])
					if newVertices:
						roofModel = roof.roofBuilder(newVertices, pitchAngle, height)
					else:
						roofModel = roof.roofBuilder(vertices, pitchAngle, height)
					roofModel = T([3])([nStorey*3/zfactor])(roofModel)
					roofModel = S([1,2,3])([xfactor*1.09,yfactor*1.09,zfactor])(roofModel)
					roofModel = T([1,2])([-SIZE([1])(roofModel)[0]*0.05,-SIZE([2])(roofModel)[0]*0.05]) (roofModel)

					#building full house model with windows and doors
					fullHouse = []
					for story in range(nStorey):
						houseModel = house.build_house(story, windowModel, doorModel, ladderBOX)
						fullHouse.append(houseModel)
						fullHouse.append(T([3])([3]))
					fullHouse = STRUCT(fullHouse)

					#returning the result
					return STRUCT([roofModel, ladderModel, fullHouse])

				return renderLadder

			return renderRoof

		return renderDoors

	return renderWindows
```

##Results

Below are shown some results, obtained with two different executions of the main function, the actual parameters of the function are the same, but the .lines used to generate all the house model are deeply different.

Actual parameter used: 
> 2 storey, 3m tall external and internal wall, $\pi$/5 pitch angle, 3m tall roof, 3m height stair, 0,2m interstep height, 0.04m riser.

##Cottage 1
<table>
 <tbody>
    <tr>
        <td><img src="img/casa1.png" style="width: 400px; height: 300px"></td>
        <td><img src="img/casa2.png" style="width: 400px; height: 300px"></td>
    </tr>
    <tr>
        <td><img src="img/casa4.png" style="width: 400px; height: 300px"></td>
        <td><img src="img/casa6.png" style="width: 400px; height: 300px"></td>
    </tr>
</tbody>
</table>

##Cottage 2

<table>
 <tbody>
    <tr>
        <td><img src="img/casa7.png" style="width: 400px; height: 300px"></td>
        <td><img src="img/casa8.png" style="width: 400px; height: 300px"></td>
    </tr>
    <tr>
        <td><img src="img/casa9.png" style="width: 400px; height: 300px"></td>
        <td><img src="img/casa10.png" style="width: 400px; height: 300px"></td>
    </tr>
</tbody>
</table>